In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
# Let's build a toy dataset
x = [[1,2],
     [3,4],
     [5,6],
     [7,8]]
y = [[3],
     [7],
     [11],
     [15]]

In [3]:
# convert the input and output from list to tensor objects
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
# Device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
X = X.to(device)
Y = Y.to(device)

In [6]:
class EniangDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    
        

In [7]:
ds = EniangDataset(X, Y)

/var/folders/y5/5kc5fqvn2tncnl_868zg4zhr0000gn/T/ipykernel_66512/704419500.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
/var/folders/y5/5kc5fqvn2tncnl_868zg4zhr0000gn/T/ipykernel_66512/704419500.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [8]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [9]:
# to see how it works
for x, y in dl:
    print(x, y)

tensor([[1., 2.],
        [3., 4.]]) tensor([[3.],
        [7.]])
tensor([[7., 8.],
        [5., 6.]]) tensor([[15.],
        [11.]])


In [10]:
class EniangNeuralNet(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.input_layer = nn.Linear(2, 8)
        self.activation_function = nn.ReLU()
        self.output_layer = nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation_function(x)
        x = self.output_layer(x)
        return x 
    

In [11]:
eniang_net = EniangNeuralNet().to(device)
loss_func = nn.MSELoss()


In [12]:
# optimization
from torch.optim import SGD
optimizer = SGD(eniang_net.parameters(), lr=0.001)

In [14]:
import time
loss_history = []
start = time.time()
for _ in range(50):
    for data in dl:
        x, y = data
        optimizer.zero_grad()
        loss_value = loss_func(eniang_net(x), y)
        loss_value.backward()
        optimizer.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.14690399169921875


#### predicting on new data points

In [15]:
# create new data point
val_x = [[10, 11]]

# convert it to tensors
val_x = torch.tensor(val_x).float().to(device)

In [16]:
# pass this new tensor to eniang_net to make predictions
eniang_net(val_x)

tensor([[20.9356]], grad_fn=<AddmmBackward0>)